<h1 style="font-size:30px;">Artistic Image Filters using OpenCV</h1>


This notebook will demonstrate various image manipulation techniques using OpenCV to mimic popular photoshop or instagram filters. In this notebook we will experiment with a wide range of filters, many of which produce artistic renderings of the original image. As you will see in the notebook, many of these effects require some experimentation, and the results for a given filter can vary quite a bit depending on the particular image used. 

<br>
<center>
<img src="https://opencv.org/wp-content/uploads/2021/09/c0-m08-04-feature-image.png" width='1000'>
</center>
<br>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

if 'google.colab' in str(get_ipython()):
    print("Downloading Code to Colab Environment")
    !wget https://www.dropbox.com/sh/287dmicr7k30m3q/AADpOe7w4ouuMp2cyA1uNFjva?dl=1 -O module-code.zip -q --show-progress
    !unzip -qq module-code.zip
    %cd Applications/
else:
    pass

### <font style='color:rgb(50,120,230)'>Load sample images</font>

In [ ]:
flower    = cv2.imread('Flowers.jpg')
house     = cv2.imread('House.jpg')
monument  = cv2.imread('Monument.jpg')
santorini = cv2.imread('Santorini.jpg')
new_york  = cv2.imread('New_York.jpg')
coast     = cv2.imread('California_Coast.jpg')

### <font style='color:rgb(50,120,230)'>Function to display original and filtered images</font>

In [ ]:
def plot(img1, img2):
    
    fig = plt.figure(figsize = (20,10))
    plt.subplot(1,2,1)
    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Original Image")

    plt.subplot(1,2,2)
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Filtered Image")

    plt.show()

# 1.  Black and White Filter

In [ ]:
def bw_filter(img):
    img_gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    return img_gray

In [ ]:
img = flower
img_bw = bw_filter(img)
plot(img, img_bw)

In [ ]:
img = new_york
img_bw = bw_filter(img)
plot(img, img_bw)

# 2. Sepia / Vintage Filter

## Matrix Transformation
**`transform()`** performs the matrix transformation of every array element.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">Function Syntax </font>
``` python
dst = cv2.transform(src, m[, dst])	
```
`dst`: output array of the same size and depth as src.

The function has **2 required input argument**:

1. `src` input array.
2. `m` transformation 2x2 or 2x3 floating-point matrix.


### <font color="green">OpenCV Documentation</font>

[**`transform()`**](https://docs.opencv.org/4.5.2/d2/de8/group__core__array.html#ga393164aa54bb9169ce0a8cc44e08ff22) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

In [ ]:
def sepia(img):
    img_sepia = img.copy()
    # Converting to RGB as sepia matrix below is for RGB.
    img_sepia = cv2.cvtColor(img_sepia, cv2.COLOR_BGR2RGB) 
    img_sepia = np.array(img_sepia, dtype = np.float64)
    img_sepia = cv2.transform(img_sepia, np.matrix([[0.393, 0.769, 0.189],
                                                    [0.349, 0.686, 0.168],
                                                    [0.272, 0.534, 0.131]]))
    # Clip values to the range [0, 255].
    img_sepia = np.clip(img_sepia, 0, 255)
    img_sepia = np.array(img_sepia, dtype = np.uint8)
    img_sepia = cv2.cvtColor(img_sepia, cv2.COLOR_RGB2BGR)
    return img_sepia

In [ ]:
img = flower
img_sepia = sepia(img)
plot(img, img_sepia)

# 3. Vignette Effect 

The vignette effect is achieved by creating a broad 2D Gaussian kernel.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">Function Syntax </font>
``` python
retval = cv2.getGaussianKernel(ksize, sigma[, ktype])

```

The function has **2 required arguments**:

1. `ksize` Aperture size. It should be odd ( `𝚔𝚜𝚒𝚣𝚎 mod 2 = 1` ) and positive.
2. `sigma` Gaussian standard deviation. If it is non-positive, it is computed from ksize as `sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8`.


### <font style="color:rgb(8,133,37)">OpenCV Documentation</font>

[**`getGaussianKernel()`**](https://docs.opencv.org/4.5.2/d4/d86/group__imgproc__filter.html#gac05a120c1ae92a6060dd0db190a61afa)

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

In [ ]:
def vignette(img, level = 2):
    
    height, width = img.shape[:2]  
    
    # Generate vignette mask using Gaussian kernels.
    X_resultant_kernel = cv2.getGaussianKernel(width, width/level)
    Y_resultant_kernel = cv2.getGaussianKernel(height, height/level)
        
    # Generating resultant_kernel matrix.
    kernel = Y_resultant_kernel * X_resultant_kernel.T 
    mask = kernel / kernel.max()
    
    img_vignette = np.copy(img)
        
    # Applying the mask to each channel in the input image.
    for i in range(3):
        img_vignette[:,:,i] = img_vignette[:,:,i] * mask
    
    return img_vignette

In [ ]:
img = flower
img_vignette = vignette(img)
plot(img, img_vignette)

In [ ]:
img = img_sepia
img_vignette = vignette(img)
plot(img, img_vignette)

# 4. Edge Detection Filter


### <font style="color:rgb(8,133,37)">OpenCV Documentation</font>

[**`cv2.Canny()`**](https://docs.opencv.org/4.5.2/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de)

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

In [ ]:
img = coast
img_edges = cv2.Canny(img, 100, 200)
plot(img, img_edges)

In [ ]:
img = coast
img_blur = cv2.GaussianBlur(img, (5,5), 0, 0)
img_edges = cv2.Canny(img_blur, 100, 200)
plot(img, img_edges)

# 5. Embossed Edges

## Convolving Images with a Kernel

`filter2D()` convolves an image with the kernel. The function applies an arbitrary linear filter to an image.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">OpenCV Documentation</font>

[**`filter2D()`**](https://docs.opencv.org/4.5.2/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04)

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

In [ ]:
def embossed_edges(img):
    
    kernel = np.array([[0, -3, -3], 
                       [3,  0, -3], 
                       [3,  3,  0]])
    
    img_emboss = cv2.filter2D(img, -1, kernel=kernel)
    return img_emboss

In [ ]:
img = house
img_emboss = embossed_edges(img)
plot(img, img_emboss)

# 6. Improving Exposure



`convertScaleAbs()` scales, calculates absolute values, and converts the result to 8-bit.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">Function Syntax</font>
``` python
dst = cv2.convertScaleAbs(src[, dst[, alpha[, beta]]])
```	
`dst`: output array.

The function has **1 required input argument:**

1. `src`: input array.

Optional arguments:
    
`alpha`: optional scale factor.
    
`beta`: optional delta added to the scaled values.

### <font style="color:rgb(8,133,37)">OpenCV Documentation</font>

[**`convertScaleAbs()`**](https://docs.opencv.org/4.5.2/d2/de8/group__core__array.html#ga3460e9c9f37b563ab9dd550c4d8c4e7d)

<hr style="border:none; height: 4px; background-color:#D3D3D3" />                     

In [ ]:
def bright(img, level):
    img_bright = cv2.convertScaleAbs(img, beta = level)
    return img_bright

In [ ]:
img = monument
img_bright = bright(img, 25)
plot(img, img_bright)

# 7. Outline Filter

In [ ]:
def outline(img, k = 9):
    
    k = max(k,9)
    kernel = np.array([[-1, -1, -1],
                       [-1,  k, -1],
                       [-1, -1, -1]])
    
    img_outline = cv2.filter2D(img, ddepth = -1, kernel = kernel)

    return img_outline

In [ ]:
img = monument
img_outline = outline(img, k = 10)
plot(img, img_outline)

In [ ]:
img = monument
img = bw_filter(img)
img_outline = outline(img, k = 10)
plot(img, img_outline)

In [ ]:
img = house
img_outline = outline(img, k = 10)
plot(img, img_outline)

# 8. Pencil Sketch Filter


**`pencilSketch()`** Creates a pencil-like non-photorealistic line drawing.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">Function Syntax </font>
``` python
dst1, dst2 = cv2.pencilSketch(src[, dst1[, dst2[, sigma_s[, sigma_r[, shade_factor]]]]])	
```

`dst1`:	Output 8-bit 1-channel image.<br>
`dst2`:	Output image with the same size and type as src.

The function has **1 required input argument**:

1. `src` Input 8-bit 3-channel image.

Optional arguments:

1. `sigma_s`:	Range between 0 to 200.
2. `sigma_r`:	Range between 0 to 1.
3. `shade_factor`:	Range between 0 to 0.1.

### <font color="green">OpenCV Documentation</font>

[**`pencilSketch()`**](https://docs.opencv.org/4.5.2/df/dac/group__photo__render.html#gae5930dd822c713b36f8529b21ddebd0c) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**Note**: Blurring is often recommended as a pre-processing step to smooth detail in the image and results in a softer and often more pleasing pencil sketch.

In [ ]:
img = flower
img_blur = cv2.GaussianBlur(img, (5,5), 0, 0)
img_sketch_bw, _ = cv2.pencilSketch(img_blur)
plot(img, img_sketch_bw)

In [ ]:
img = santorini
img_blur = cv2.GaussianBlur(img, (5,5), 0, 0)
img_sketch_bw, img_sketch_color = cv2.pencilSketch(img_blur)
plot(img, img_sketch_bw)

# 9. Stylization Filter

**`stylization()`** aims to produce digital imagery with a wide variety of effects not focused on photorealism. Edge-aware filters are ideal for stylization, as they can abstract regions of low contrast while preserving, or enhancing, high-contrast features.

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

### <font style="color:rgb(8,133,37)">Function Syntax </font>
``` python
dst = cv2.stylization(src[, dst[, sigma_s[, sigma_r]]])

```
`dst`: output array of the same size and depth as src.

The function has **1 required input argument**:

1. `src` Input 8-bit 3-channel image.

Optional arguments:

1. `sigma_s`:	Range between 0 to 200.
2. `sigma_r`:	Range between 0 to 1.


### <font color="green">OpenCV Documentation</font>

[**`stylization()`**](https://docs.opencv.org/4.5.2/df/dac/group__photo__render.html#gacb0f7324017df153d7b5d095aed53206) <br>

<hr style="border:none; height: 4px; background-color:#D3D3D3" />

**Note**: Blurring is often recommended as a pre-processing step to smooth detail in the image and results in a softer and often more pleasing result.

In [ ]:
img = santorini
img_blur = cv2.GaussianBlur(img, (5,5), 0, 0)
img_style = cv2.stylization(img_blur, sigma_s = 40, sigma_r = 0.1)
plot(img, img_style)